<a href="https://colab.research.google.com/github/BrunoAlexandreAmaral/Spark/blob/main/INSTALANDO_SPARKUI_E_SPARKSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [45]:
!pip install pyngrok

In [27]:
import pandas as pq
import numpy as np
from google.colab import files
uploaded = files.upload()

In [48]:
from pyngrok import ngrok

In [29]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip yes

--2025-03-04 15:22:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  16.9MB/s    in 0.8s    

2025-03-04 15:22:22 (16.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
caution: filename not matched:  yes


In [30]:
from pyspark.sql import SparkSession

In [31]:
spark = SparkSession.builder.config("spark.ui.port","4050").appName("programa").getOrCreate()

In [34]:
!./ngrok config add-authtoken "SEU AUTTOKEN DO ngrok"

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [50]:
!./ngrok authtoken "SEU AUTTOKEN DO ngrok"
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10s
!curl -s http://localhost:4050/api/tunnels | grep -Po  'public_url":"(?=https)\K[^"]"'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [53]:
ngrok_auth_token = "SEU AUTTOKEN DO ngrok"
get_ipython().system_raw('./ngrok authtoken $ngrok_auth_token && ./ngrok tcp 4050 &')
!sleep 10
print(ngrok.connect(4050))

NgrokTunnel: "https://6c86-34-125-56-157.ngrok-free.app" -> "http://localhost:4050"


In [52]:
ngrok.set_auth_token("SEU AUTTOKEN DO ngrok")

In [54]:
caminho = "/content/pix_transactions.json"

In [93]:
df = spark.read.json(caminho, schema=tipos, timestampFormat="yyyy-MM-dd")

In [94]:
df.printSchema()

root
 |-- transaction_date: timestamp (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- fraude: string (nullable = true)
 |-- id_transacao: string (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



In [61]:
from pyspark.sql.types import StructType,StructField,StringType, DoubleType, IntegerType, TimestampType

In [70]:
from pyspark.sql.functions import to_timestamp

In [95]:
df.show()

+-------------------+---------+--------------------+------+------------+--------------------+--------+
|   transaction_date|chave_pix|        destinatario|fraude|id_transacao|           remetente|   valor|
+-------------------+---------+--------------------+------+------------+--------------------+--------+
|2022-03-19 00:00:00|      cpf|{Itau, Gabriel Cu...|     0|        1000|{BTG, Jonathan Go...|    7.05|
|2021-01-26 00:00:00|aleatoria|{XP, Diego Souza,...|     0|        1001|{BTG, Jonathan Go...|   37.28|
|2022-05-31 00:00:00|aleatoria|{BTG, Nicole Nune...|     0|        1002|{BTG, Jonathan Go...|  282.73|
|2022-07-04 00:00:00|aleatoria|{Nubank, Maria Fe...|     0|        1003|{BTG, Jonathan Go...| 8447.92|
|2021-09-11 00:00:00|aleatoria|{C6, Isabel Silva...|     0|        1004|{BTG, Jonathan Go...|   58.51|
|2022-02-11 00:00:00|  celular|{Caixa, Anthony C...|     0|        1005|{BTG, Jonathan Go...| 6655.12|
|2022-05-10 00:00:00|      cpf|{Itau, Eloah Mont...|     0|        1006|{

In [92]:
tipos = StructType([
    StructField("transaction_date", TimestampType()),
    StructField("chave_pix", StringType()),
    StructField("destinatario", tipo_dois),
    StructField("fraude", StringType()),
    StructField("id_transacao", StringType()),
    StructField("remetente", tipo_dois),
    StructField("valor", DoubleType()),
])

In [91]:
tipo_dois = StructType([StructField("banco", StringType()),
                        StructField("nome", StringType()),
                        StructField("tipo", StringType())])

In [98]:
spark.read.json(caminho, schema=tipos, timestampFormat="yyyy-MM-dd").createOrReplaceTempView("transacoes")

In [100]:
spark.sql("select * from transacoes limit 10").show()

+-------------------+---------+--------------------+------+------------+--------------------+-------+
|   transaction_date|chave_pix|        destinatario|fraude|id_transacao|           remetente|  valor|
+-------------------+---------+--------------------+------+------------+--------------------+-------+
|2022-03-19 00:00:00|      cpf|{Itau, Gabriel Cu...|     0|        1000|{BTG, Jonathan Go...|   7.05|
|2021-01-26 00:00:00|aleatoria|{XP, Diego Souza,...|     0|        1001|{BTG, Jonathan Go...|  37.28|
|2022-05-31 00:00:00|aleatoria|{BTG, Nicole Nune...|     0|        1002|{BTG, Jonathan Go...| 282.73|
|2022-07-04 00:00:00|aleatoria|{Nubank, Maria Fe...|     0|        1003|{BTG, Jonathan Go...|8447.92|
|2021-09-11 00:00:00|aleatoria|{C6, Isabel Silva...|     0|        1004|{BTG, Jonathan Go...|  58.51|
|2022-02-11 00:00:00|  celular|{Caixa, Anthony C...|     0|        1005|{BTG, Jonathan Go...|6655.12|
|2022-05-10 00:00:00|      cpf|{Itau, Eloah Mont...|     0|        1006|{BTG, Jona